# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [38]:
""" Import all the necessary python modules necessary for the project """
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [39]:
""" First check the present working directory """
print(os.getcwd())

""" Get current folder and subfolder for event data """
filepath = os.getcwd() + '/event_data'

""" Executing a for loop to fetch all the associated file path from the directory"""
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [40]:
full_data_rows_list = [] 
    
""" Reading the CSV file and appending it into the list for every filepath in filepath list """
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)      
        for line in csvreader:
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [41]:
""" Code to check the number of lines in the CSV """
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [42]:
""" We will use cassandra library to connect to our instance """

from cassandra.cluster import Cluster
cluster = Cluster()

""" Creating session to establish connection and for executing queries """
session = cluster.connect()

#### Create Keyspace

In [43]:
""" In this step, we will create a keyspace called sparkifydb in cassandra """
try:
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS sparkifydb \
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor': 1} """)
except Exception as e:
    print("ERROR while creating the database")
    print (e)

#### Set Keyspace

In [44]:
""" In this step, we will set our newly created keyspace for further development """
try:
    session.set_keyspace("sparkifydb")
except Exception as e:
    print(" ERROR while connecting to the database")
    print (e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [45]:
""" In this step we will create table for song_library based on artist,song title, track duration for each session\
We have used session_id as our partition column and item_in_session as our clustering column\
to get fetch data in sorted order """

try:
    session.execute(""" CREATE TABLE IF NOT EXISTS song_library_session \
    (session_id int, item_in_session int,artist text, song_title text, song_length float, \
    PRIMARY KEY (session_id, item_in_session)) """)
except Exception as e:
    print(" ERROR while creating table")
    print (e)


In [46]:
""" This step will read event_datafile_new csv, select particular columns as per schema and insert data into the select table"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = " INSERT INTO song_library_session (session_id, item_in_session,artist, song_title, song_length) "
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [47]:
""" This query is to fetch data of artist, with thier tracks , length of the track for particular session,\
based on item_in_session and session_id"""

try:
    rows = session.execute(""" select artist, song_title, song_length from song_library_session\
            where session_id = 338 and item_in_session = 4 """)
    for row in rows:
        print("Artist: "+row.artist,"Song_title: "+row.song_title, "Song_lenght: "+str(row.song_length))
except Exception as e:
    print("ERROR occured while fetching data")
    print (e)

Artist: Faithless Song_title: Music Matters (Mark Knight Dub) Song_lenght: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [48]:
""" In this step we will create table for user based on firstname, lastname and song track.\
We have used user_id and session_id as our partition column \
and item_in_session as our clustering column to get fetch data in sorted order """

try:
    session.execute(""" CREATE TABLE IF NOT EXISTS song_playlist_session \
    (user_id int,session_id int, item_in_session int,artist text, song_title text,firstName text, lastName text, \
    PRIMARY KEY ((user_id,session_id), item_in_session)) """)
except Exception as e:
    print(" ERROR while creating table")
    print (e)        

In [49]:
""" This step will read event_datafile_new csv, select particular columns as per schema and insert data into the select table"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = " INSERT INTO song_playlist_session \
        (user_id,session_id, item_in_session,artist, song_title, firstName, lastName ) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [50]:
""" This query is to fetch data for users, who listen to particular artist and songs of that artist for particular session,\
based on user_id and session_id"""

try:
    rows = session.execute(""" select artist, song_title,firstName, lastName from song_playlist_session \
                            where user_id = 10 and session_id = 182 """)
    for row in rows:
         print("artist: "+row.artist,"song_title: "+row.song_title,"first_name: "+row.firstname,"last_name: "+row.lastname)
except Exception as e:
    print("ERROR occured while fetching data")
    print (e)

artist: Down To The Bone song_title: Keep On Keepin' On first_name: Sylvie last_name: Cruz
artist: Three Drives song_title: Greece 2000 first_name: Sylvie last_name: Cruz
artist: Sebastien Tellier song_title: Kilometer first_name: Sylvie last_name: Cruz
artist: Lonnie Gordon song_title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) first_name: Sylvie last_name: Cruz


In [51]:
""" In this step we will create table for user based on firstname, lastname and song track.\
We have used song_title as our partition column and user_id as our clustering column for fast retrivals """
try:
    session.execute(""" CREATE TABLE IF NOT EXISTS users_song_history \
    (song_title text, user_id int, firstName text, lastName text, \
    PRIMARY KEY (song_title, user_id) )""")
except Exception as e:
    print(" ERROR while creating table")
    print (e) 
                    

In [52]:
""" This step will read event_datafile_new csv, select particular columns as per schema and insert data into the select table"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = " INSERT INTO users_song_history (song_title, user_id, firstName, lastName ) "
        query = query + "VALUES (%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [53]:
""" In this query we want users first name and lastname, who have listen to song_track 'All Hands Against His Own'."""

try:
    rows = session.execute(""" select firstName,lastName from users_song_history \
    where song_title = 'All Hands Against His Own' """)
    for row in rows:
        print("first_name: "+row.firstname,"last_name: "+row.lastname)
except Exception as e:
    print("ERROR occured while fetching data")
    print (e)

first_name: Jacqueline last_name: Lynch
first_name: Tegan last_name: Levine
first_name: Sara last_name: Johnson


### Drop the tables before closing out the sessions

In [37]:
""" This step will drop the all the tables in the given keyspace"""
try:
    session.execute(""" DROP TABLE song_library_session """)
except Exception as e:
    print("ERROR occured while dropping table song_library_session")
    print (e)

try:
    session.execute(""" DROP TABLE song_playlist_session  """)
except Exception as e:
    print("ERROR occured while dropping table song_playlist_session")
    print (e)

try:
    session.execute(""" DROP TABLE users_song_history  """)
except Exception as e:
    print("ERROR occured while dropping table users_song_history")
    print (e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()